# Experiment 003: Pre-optimized Baseline with Multi-Phase Optimization

Using chistyakov/submission_best.csv as starting point (score: 70.93, no overlaps).
Target: 68.92 | Gap: only 2.01 points!

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar

getcontext().prec = 30
scale_factor = Decimal('1e15')

print("Libraries loaded")

Libraries loaded


In [2]:
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / scale_factor

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_tree_list_side_length(trees)
    return float(side ** 2 / Decimal(n))

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def load_configuration_from_df(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

def score_submission(file_path, max_n=200):
    df = pd.read_csv(file_path)
    total_score = 0.0
    failed_overlap_n = []
    
    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            current_score = get_score(trees, n)
            total_score += current_score
            if has_overlap(trees):
                failed_overlap_n.append(n)
    
    return total_score, failed_overlap_n

print("Helper functions defined")

Helper functions defined


In [4]:
# Verify the pre-optimized baseline
print("Scoring chistyakov/submission_best.csv baseline...")
baseline_score, baseline_overlaps = score_submission('/home/code/experiments/003_preoptimized/baseline.csv')
print(f"Baseline score: {baseline_score:.6f}")
print(f"Overlaps: {baseline_overlaps if baseline_overlaps else 'None'}")
print(f"Gap to target (68.92): {baseline_score - 68.922808:.6f} points")

Scoring chistyakov/submission_best.csv baseline...


Baseline score: 70.926150
Overlaps: None
Gap to target (68.92): 2.003342 points


In [5]:
# Also check other pre-optimized baselines for comparison
print("\nChecking other pre-optimized baselines...")

baselines = [
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/71.97.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/72.49.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv',
]

import os
for path in baselines:
    if os.path.exists(path):
        try:
            score, overlaps = score_submission(path)
            print(f"{os.path.basename(path)}: score={score:.6f}, overlaps={len(overlaps)}")
        except Exception as e:
            print(f"{os.path.basename(path)}: Error - {e}")
    else:
        print(f"{os.path.basename(path)}: Not found")


Checking other pre-optimized baselines...


santa-2025.csv: score=70.676102, overlaps=12


71.97.csv: score=71.972027, overlaps=0


72.49.csv: score=72.495739, overlaps=0


ensemble.csv: score=70.676102, overlaps=13


In [6]:
# Try to repair santa-2025.csv overlaps using chistyakov configurations
print("Repairing santa-2025.csv overlaps...")

# Load both CSVs
df_santa = pd.read_csv('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv')
df_chistyakov = pd.read_csv('/home/code/experiments/003_preoptimized/baseline.csv')

# Find which N values have overlaps in santa-2025.csv
overlap_n = []
for n in range(1, 201):
    trees = load_configuration_from_df(n, df_santa)
    if trees and has_overlap(trees):
        overlap_n.append(n)

print(f"Overlapping N values in santa-2025.csv: {overlap_n}")

# Replace overlapping configurations with chistyakov configurations
df_repaired = df_santa.copy()

for n in overlap_n:
    # Remove old rows for this N
    mask = df_repaired['id'].str.startswith(f'{n:03d}_')
    df_repaired = df_repaired[~mask]
    
    # Add rows from chistyakov
    chistyakov_rows = df_chistyakov[df_chistyakov['id'].str.startswith(f'{n:03d}_')]
    df_repaired = pd.concat([df_repaired, chistyakov_rows], ignore_index=True)

# Sort by id
df_repaired = df_repaired.sort_values('id').reset_index(drop=True)

# Save repaired CSV
df_repaired.to_csv('/home/code/experiments/003_preoptimized/repaired_baseline.csv', index=False)

# Score the repaired version
repaired_score, repaired_overlaps = score_submission('/home/code/experiments/003_preoptimized/repaired_baseline.csv')
print(f"\nRepaired baseline score: {repaired_score:.6f}")
print(f"Overlaps: {repaired_overlaps if repaired_overlaps else 'None'}")
print(f"Gap to target: {repaired_score - 68.922808:.6f} points")

Repairing santa-2025.csv overlaps...


Overlapping N values in santa-2025.csv: [30, 41, 43, 61, 77, 110, 114, 117, 134, 137, 170, 171]



Repaired baseline score: 70.682741
Overlaps: None
Gap to target: 1.759933 points
